In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pprint import pprint
from datetime import datetime

from IPython.display import display, Markdown

import utils
import prompts

from prompts import Job_Post, Resume_Builder

In [ ]:
## Inputs
my_files_dir = "my_applications"
job_file = "square.job"
raw_resume_file = "resume_raw.yaml"

# open_ai model. gpt-4 can be 20-30 times costlier than the default model gpt-3.5-turbo
openai_model_name = "gpt-4"
openai_model_name = "gpt-3.5-turbo"

In [ ]:
# Step 1 - Read and parse job posting
job_post = Job_Post(
    utils.read_jobfile(os.path.join(my_files_dir, job_file)),
    temperature=0.3,
    model_name=openai_model_name,
)
parsed_job = job_post.parse_job_post()
utils.write_yaml(parsed_job)

In [ ]:
# Step 2 - read raw resume and create Resume builder object
my_resume = Resume_Builder(
    raw_resume=utils.read_yaml(os.path.join(my_files_dir, raw_resume_file)),
    job_post=job_post,
    temperature=0,
    model_name=openai_model_name,
)
projects = my_resume.projects_raw
experiences = my_resume.experiences_raw
skills = my_resume.skills_raw

In [ ]:
# Step 3 - Rewrite sections as needed
# Comment out any lines for sections that do not need to be revised
experiences = my_resume.rewrite_unedited_experiences(verbose=False)
projects = my_resume.rewrite_section(section=projects, verbose=False)

pprint(
    dict(experiences=experiences, projects=projects),
    indent=1,
    width=80,
    sort_dicts=False,
)

In [ ]:
# Step 4 - Review the generated output in previous cell.
# If any updates are needed, copy the corresponding variables and make edits
# Leave this cell blank if no edits are needed


In [ ]:
# Step 6 - Extract skills
my_resume.experiences = experiences
my_resume.projects = projects

skills = my_resume.extract_skills(verbose=False)
pprint(
    dict(skills=skills, projects=my_resume.projects, experiences=my_resume.experiences),
    indent=1,
    width=80,
    sort_dicts=False,
)

In [ ]:
# Step 7 - Make any changes to skills, projects, or experiences by copying them from previous cell's output
# and editing as needed. Leave this cell blank if no changes are needed.


In [ ]:
# Step 8 - Identify resume improvements
my_resume.experiences = experiences
my_resume.skills = skills
my_resume.projects = projects

improvements = my_resume.suggest_improvements(verbose=False)
pprint(improvements, indent=1, width=100, sort_dicts=False)

In [ ]:
# Step 9 - Update experience, skills, projects from Step 6/7 based on suggested improvements
# Leave this cell blank if no edits are needed.


In [ ]:
# Step 10 - Create a resume summary
my_resume.experiences = experiences
my_resume.skills = skills
my_resume.projects = projects

summary = my_resume.create_summary(verbose=False)
pprint(
    dict(
        summary=summary,
        experience=my_resume.experiences,
        projects=my_resume.projects,
        skills=skills,
    ),
    indent=1,
    width=80,
    sort_dicts=False,
)

In [ ]:
# Step 11 - Make any changes to summary, experience, projects, skills by copying it from previous cell's output
# and editing as needed. Leave this cell blank if no changes are needed.


In [ ]:
# Step 12 - Generate final resume yaml for review
my_resume.summary = summary
my_resume.experiences = experiences
my_resume.projects = projects
my_resume.skills = skills

company_name = my_resume.job_post.parsed_job["company"]
job_title = my_resume.job_post.parsed_job["job_title"]
today_date = datetime.today().strftime("%Y%m%d")
resume_filename = os.path.join(
    my_files_dir, f"{today_date}__{company_name}__{job_title}"
)
resume_final = my_resume.finalize()
print(f"#filename: {resume_filename}.yaml\n")
utils.write_yaml(resume_final, filename=f"{resume_filename}.yaml")

In [ ]:
# Generate pdf from yaml - you can directly edit the yaml file as needed, and re-run this cell
pdf_file = utils.generate_pdf(yaml_file=f"{resume_filename}.yaml")
display(Markdown((f"[{pdf_file}](<{pdf_file}>)")))